In [1]:
import os

In [2]:
%pwd

'/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main'

In [312]:
%pip install mlflow dagshub

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached sqlparse-0.5.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached Mako-1.3.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached google_auth-2.35.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached zipp-3.20.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 35.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 38.4 MB/s eta 0:00:00
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached Markdown-3.7-py3-none-any.whl (106 kB)
   ━━━━━━━━

In [313]:
import mlflow
import dagshub
dagshub.init(repo_owner='vicky5848d', repo_name='Kidney-Disease-Classification', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=09103e76-e40b-4955-bcc3-a30797be3cff&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ca335d9c7aec9c9df4c795e9809c19f8a6df4af04d79d1657d60b417a30521ea




[2024-11-05 01:04:17,021: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2024-11-05 01:04:17,186: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2024-11-05 01:04:17,340: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


...

[2024-11-05 01:04:17,344: INFO: helpers: Accessing as vicky5848d]
[2024-11-05 01:04:17,537: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/vicky5848d/Kidney-Disease-Classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "vicky5848d/Kidney-Disease-Classification"

[2024-11-05 01:04:17,542: INFO: helpers: Initialized MLflow to track repo "vicky5848d/Kidney-Disease-Classification"]


Repository vicky5848d/Kidney-Disease-Classification initialized!

[2024-11-05 01:04:17,544: INFO: helpers: Repository vicky5848d/Kidney-Disease-Classification initialized!]


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [314]:
%pip install torcheval
import os
import urllib.request as request
from zipfile import ZipFile
import cv2
import pandas as pd
import numpy as np
# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision
from torcheval.metrics import MulticlassAUROC

# Sklearn Imports
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

Note: you may need to restart the kernel to use updated packages.


In [4]:
CONFIG = {
    "seed": 42,
    "epochs": 50,
    "img_size": 384,
    "model_name": "tf_efficientnet_b0_ns",
    "checkpoint_path" : None,
    "train_batch_size": 12,
    "valid_batch_size": 10,
    "learning_rate": 1e-3,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-8,
    "T_max": 500,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 3,
    "device": "cpu",
}

import yaml
print(yaml.dump(CONFIG))

T_max: 500
checkpoint_path: null
device: cpu
epochs: 50
fold: 0
img_size: 384
learning_rate: 0.001
min_lr: 1.0e-08
model_name: tf_efficientnet_b0_ns
n_accumulate: 3
n_fold: 5
scheduler: CosineAnnealingLR
seed: 42
train_batch_size: 12
valid_batch_size: 10
weight_decay: 1.0e-06



In [342]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [317]:
pwd

'/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main'

In [318]:
from sklearn.model_selection import train_test_split

In [ ]:
## TO DO
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)


In [ ]:

path= "/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main/artifacts/data_ingestion/kidneyData.csv"

df = pd.read_csv(path)
df_cysts = df[df["target"] == 0].reset_index(drop=True)
df_normal = df[df["target"] == 1].reset_index(drop=True)
df_stone = df[df["target"] == 2].reset_index(drop=True)
df_tumor = df[df["target"] == 3].reset_index(drop=True)

df_target = df["target"].reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(

    df, df_target, test_size=0.33, random_state=int(os.environ['PYTHONHASHSEED']))


X_train, X_valid, y_train, y_valid = train_test_split(

    X_train, y_train, test_size=0.25, random_state=int(os.environ['PYTHONHASHSEED']))

In [320]:
df_normal.shape, df_tumor.shape, df_stone.shape, df_cysts.shape

((5077, 6), (2283, 6), (1377, 6), (3709, 6))

In [321]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6253, 6), (4108, 6), (6253,), (4108,))

In [1]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

NameError: name 'A' is not defined

In [344]:
## DATA LOADER
class KidneyDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['path'].values   
        self.targets = df['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img_path = "/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main/artifacts/data_ingestion/" + img_path

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }

In [3]:
import yaml
yaml.dump(data_transforms, "tranform.yaml")

NameError: name 'data_transforms' is not defined

In [ ]:
t = KidneyDataset(X_train, transforms=data_transforms["train"])

In [331]:
%pwd

'/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main'

In [332]:
t[1]

{'image': tensor([[[-2.1179, -2.1179, -2.1179,  ..., -1.7240, -1.5185, -1.1075],
          [-2.1179, -2.1179, -2.1179,  ..., -1.6042, -1.3815, -1.0562],
          [-2.1179, -2.1179, -2.1179,  ..., -1.3130, -1.2274, -1.1075],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -1.6331, -1.4230, -1.0028],
          [-2.0357, -2.0357, -2.0357,  ..., -1.5105, -1.2829, -0.9503],
          [-2.0357, -2.0357, -2.0357,  ..., -1.2129, -1.1253, -1.0028],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.4036, -1.1944, -0.7761],
          [-1.8044,

In [345]:
## Reference : https://amaarora.github.io/posts/2020-08-30-gempool.html

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [346]:
class EffNetModel(nn.Module):
    def __init__(self, model_name, num_classes=4, pretrained=True, checkpoint_path=None):
        super(EffNetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, checkpoint_path=checkpoint_path)

        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.sigmoid(self.linear(pooled_features))
        return output

    
model = EffNetModel(CONFIG['model_name'], checkpoint_path=CONFIG['checkpoint_path'])
model.to(CONFIG['device']);

[2024-11-05 01:15:09,222: INFO: _builder: Loading pretrained weights from Hugging Face hub (timm/tf_efficientnet_b0.ns_jft_in1k)]


/home/vikram/miniconda3/envs/ds/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


[2024-11-05 01:15:33,472: INFO: _hub: [timm/tf_efficientnet_b0.ns_jft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.]


In [347]:
def criterion(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)

(torch.Size([3, 5]), torch.Size([3]))

In [ ]:
from tqdm import tqdm
import gc

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    num_batches=len(dataloader)
    bar = tqdm(enumerate(dataloader), total=len(dataloader),position=0, leave=True)
    for batch, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = images.size(0)
        
        outputs = model(images)
       
        loss = criterion(outputs, targets)
            
        loss.backward()
    
        if (batch + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        multi_auroc = MulticlassAUROC(num_classes=4)
        multi_auroc.update(input=outputs, target=targets)
        auroc = multi_auroc.compute()

              
        
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        if batch % 25 == 0:
            loss, current = epoch_loss, batch
            step = (batch // 25) + epoch*num_batches//25
            mlflow.log_metric("loss", f"{loss:3f}", step=(step))
            mlflow.log_metric("MulticlassAUROC", f"{epoch_auroc:3f}", step=(step))
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss, epoch_auroc

In [ ]:
@torch.inference_mode()
def valid_one_epoch(model,optimizer, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    
   
    bar = tqdm(enumerate(dataloader), total=len(dataloader), position=0, leave=True)
    for batch, data in bar:
       
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = images.size(0)

        outputs = model(images)
        
        loss = criterion(outputs, targets)

        multi_auroc = MulticlassAUROC(num_classes=4)
        multi_auroc.update(input=outputs, target=targets)
        auroc = multi_auroc.compute()
        
        
        
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, Valid_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])   
    
  
    mlflow.log_metric("eval_loss", f"{epoch_loss:2f}", step=epoch)
    mlflow.log_metric("eval_MulticlassAUROC", f"{epoch_auroc:2f}", step=epoch)
    gc.collect()
    
    return epoch_loss, epoch_auroc

In [354]:
import time
from collections import defaultdict
import copy

def run_training(model, optimizer, scheduler, device, num_epochs, train_loader, valid_loader, run_id="latest"):
    os.makedirs("/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main/artifacts/runs/"+run_id)
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)
    multi_auroc = MulticlassAUROC(num_classes=4,average=None)
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, optimizer,valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        history['lr'].append( scheduler.get_lr()[0] )
        
        # deep copy the model
        print(f"best_epoch_auroc {best_epoch_auroc}")
        print(f"val_epoch_auroc {val_epoch_auroc}")
        
        
        if best_epoch_auroc <= val_epoch_auroc:
            print(f"Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "/AUROC{:.4f}_Loss{:.4f}_epoch{:.0f}.bin".format(val_epoch_auroc, val_epoch_loss, epoch)
            torch.save(model.state_dict(), "/home/vikram/Downloads/pop_os_backup/Kidney-Disease-Classification-Deep-Learning-Project-main/artifacts/" + run_id + PATH)
            # Save a model file from the current directory
            print(f"Model Saved{PATH}")
            
        
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [355]:
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                       weight_decay=CONFIG['weight_decay'])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])

In [356]:
train_dataset = KidneyDataset(X_train, transforms=data_transforms["train"])
valid_dataset = KidneyDataset(X_valid, transforms=data_transforms["valid"])

train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=1, shuffle=True, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=1, shuffle=False, pin_memory=True)

In [357]:
len(train_loader)

521

In [358]:
t = torch.tensor([[0.3924, 0.7158, 0.5311, 0.0579],
        [0.2794, 0.5616, 0.5216, 0.0404],
        [0.3344, 0.7559, 0.4294, 0.0984],
        [0.2147, 0.6359, 0.5156, 0.0778],
        [0.2850, 0.7352, 0.5044, 0.0943],
        [0.3265, 0.7074, 0.5846, 0.0850],
        [0.2186, 0.8068, 0.4628, 0.0655],
        [0.1892, 0.8145, 0.4243, 0.0704],
        [0.2459, 0.6203, 0.6948, 0.1244],
        [0.1527, 0.7540, 0.5675, 0.0454],
        [0.5703, 0.7642, 0.4513, 0.0628],
        [0.1669, 0.7222, 0.5182, 0.0436],
        [0.4301, 0.7556, 0.7043, 0.1060],
        [0.3571, 0.6288, 0.4984, 0.1113],
        [0.2755, 0.8199, 0.2628, 0.1294],
        [0.3336, 0.6621, 0.4585, 0.1290]])

t.shape

torch.Size([16, 4])

In [ ]:
#torch.argmax(t, dim=1).to(torch.float)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.])

In [359]:
CONFIG['learning_rate']

0.001

In [ ]:
#model, history = run_training(model, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader,
                              valid_loader=valid_loader)

[INFO] Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU



100%|██████████| 131/131 [02:09<00:00,  1.01it/s, Epoch=1, LR=1.24e-5, Valid_Auroc=tensor(0.9725), Valid_Loss=0.753]


Validation AUROC Improved (-inf ---> 0.9725459218025208)
Model SavedAUROC0.9725_Loss0.7531_epoch1.bin
Training complete in 0h 23m 41s
Best AUROC: 0.9725


In [361]:
%pip install torchinfo
from torchinfo import summary
from mlflow.data.pandas_dataset import PandasDataset
run_id = "laptop_run2"
with mlflow.start_run(run_name=run_id):
    params = {
        
        
        "img_size": CONFIG['img_size'],
        "model_name": CONFIG["model_name"],       
        "scheduler": CONFIG['scheduler'],
        "min_lr": CONFIG['min_lr'],     
        "n_accumulate": CONFIG['n_accumulate'],
        "seed": CONFIG['seed'],
        "epochs": CONFIG['epochs'],
        "learning_rate": CONFIG['learning_rate'],
        "train_batch_size": CONFIG['train_batch_size'],
        "val_batch_size": CONFIG['valid_batch_size'],
        "loss_function": "CrossEntropyLoss",
        "weight_decay": CONFIG['weight_decay'],
        "metric_function": "MulticlassAUROC",
        "optimizer": "Adam",
        "Tmax": CONFIG['T_max'], 
        "eta_min": CONFIG['min_lr']
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")
    
    X_train_dataset: PandasDataset = mlflow.data.from_pandas(X_train)
    X_valid_dataset: PandasDataset = mlflow.data.from_pandas(X_valid)
        
    mlflow.log_input(X_train_dataset, context="training")
    mlflow.log_input(X_valid_dataset, context="validation")
    
    
    mlflow.set_tag('mlflow.runName', run_id)
    
    
    
    model, history = run_training(model, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader,
                              valid_loader=valid_loader,
                              run_id=run_id)

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, "model")

Note: you may need to restart the kernel to use updated packages.


/home/vikram/miniconda3/envs/ds/lib/python3.10/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


[INFO] Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU



 17%|█▋        | 86/521 [03:29<17:37,  2.43s/it, Epoch=1, LR=0.000992, Train_Auroc=tensor(0.6129), Train_Loss=1.22]
2024/11/05 01:21:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run laptop_run2 at: https://dagshub.com/vicky5848d/Kidney-Disease-Classification.mlflow/#/experiments/0/runs/d50f34b489be48128d8c5b29e081a645.
2024/11/05 01:21:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/vicky5848d/Kidney-Disease-Classification.mlflow/#/experiments/0.


KeyboardInterrupt: 

In [ ]:
## Loading Model
model_load = EffNetModel(CONFIG['model_name'], checkpoint_path=None)
state = torch.load("/kaggle/working/AUROC0.6751_Loss0.7461_epoch1.bin", map_location=torch.device('cpu'))
model_load.load_state_dict(state)

In [ ]:

plt.plot( range(history.shape[0]), history["Train Loss"].values, label="Train Loss")
plt.plot( range(history.shape[0]), history["Valid Loss"].values, label="Valid Loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot( range(history.shape[0]), history["Train AUROC"].values, label="Train AUROC")
plt.plot( range(history.shape[0]), history["Valid AUROC"].values, label="Valid AUROC")
plt.xlabel("epochs")
plt.ylabel("AUROC")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.plot( range(history.shape[0]), history["lr"].values, label="lr")
plt.xlabel("epochs")
plt.ylabel("lr")
plt.grid()
plt.legend()
plt.show()

In [ ]:
"""
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e
    """

[2023-10-02 09:52:42,695: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-02 09:52:42,700: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-02 09:52:42,702: INFO: common: created directory at: artifacts]
[2023-10-02 09:52:42,703: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-10-02 09:52:43,136: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64

In [306]:
%pip install torcheval

Note: you may need to restart the kernel to use updated packages.


In [307]:
import torch
from torcheval.metrics import MulticlassAUROC
metric = MulticlassAUROC(num_classes=4)
input = torch.tensor([[0.1, 0.1, 0.1, 0.1], [0.5, 0.5, 0.5, 0.5], [0.7, 0.7, 0.7, 0.7], [0.8, 0.8, 0.8, 0.8]])
target = torch.tensor([0, 1, 2, 3])
metric.update(input, target)
metric.compute()

tensor(0.5000)

In [310]:
import torch
from torcheval.metrics import MulticlassAUROC
metric = MulticlassAUROC(num_classes=4, average=None)
input = torch.tensor([[0.8, 0.3, 0.1, 0.1], [0.5, 0.6, 0.5, 0.5], [0.7, 0.8, 0.7, 0.7], [0.8, 0.8, 0.96, 0.8]])
target = torch.tensor([0, 1, 2, 3])
metric.update(input, target)
metric.compute()

tensor([0.8333, 0.3333, 0.6667, 1.0000])

In [ ]:
e = 5   
b = 1000

for epoch in range(e):
    print(f"EPOCH {epoch}")
    for bat in range(b):
        if bat % 100 == 0:
           
            step = bat // 100 * (epoch + 1)
            print(f"step {step} | batch {bat}")


EPOCH 0
step 0 | batch 0
step 1 | batch 100
step 2 | batch 200
step 3 | batch 300
step 4 | batch 400
step 5 | batch 500
step 6 | batch 600
step 7 | batch 700
step 8 | batch 800
step 9 | batch 900
EPOCH 1
step 0 | batch 0
step 2 | batch 100
step 4 | batch 200
step 6 | batch 300
step 8 | batch 400
step 10 | batch 500
step 12 | batch 600
step 14 | batch 700
step 16 | batch 800
step 18 | batch 900
EPOCH 2
step 0 | batch 0
step 3 | batch 100
step 6 | batch 200
step 9 | batch 300
step 12 | batch 400
step 15 | batch 500
step 18 | batch 600
step 21 | batch 700
step 24 | batch 800
step 27 | batch 900
EPOCH 3
step 0 | batch 0
step 4 | batch 100
step 8 | batch 200
step 12 | batch 300
step 16 | batch 400
step 20 | batch 500
step 24 | batch 600
step 28 | batch 700
step 32 | batch 800
step 36 | batch 900
EPOCH 4
step 0 | batch 0
step 5 | batch 100
step 10 | batch 200
step 15 | batch 300
step 20 | batch 400
step 25 | batch 500
step 30 | batch 600
step 35 | batch 700
step 40 | batch 800
step 45 | batc

Heller, N., Sathianathen, N., Kalapara, A., Walczak, E., Moore, K., Kaluzniak, H., Rosenberg, J., Blake, P., Rengel, Z., Oestreich, M., Dean, J., Tradewell, M., Shah, A., Tejpaul, R., Edgerton, Z., Peterson, M., Raza, S., Regmi, S., Papanikolopoulos, N., Weight, C.  (2019) Data from C4KC-KiTS  [Data set]. The Cancer Imaging Archive. DOI: 10.7937/TCIA.2019.IX49E8NX 